In [1]:
! nvidia-smi -L


GPU 0: Tesla T4 (UUID: GPU-723b475b-916d-ba17-c4f0-c09a2c274a94)
GPU 1: Tesla T4 (UUID: GPU-89ae1cb7-090c-1ec7-5ecc-105225d438f6)


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/qadocs/Krista QA documents.pdf


In [3]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


In [4]:
# Install required packages
!pip install -q langchain tiktoken pypdf chromadb faiss-gpu
!pip install -q transformers InstructorEmbedding sentence_transformers
!pip install -q bitsandbytes xformers einops

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2023.58.190319 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is incompatible.
google-cloud-pubsublite 1.8.1 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.3.1 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.1.2 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
ydata-profiling 4.1.2 requires tqdm<4.65,>=4.48.2, but you have tqdm 4.65.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [5]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import textwrap

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers

# Import the necessary modules from the Transformers library
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

print(langchain.__version__)

0.0.209


In [6]:
# ========== Packages ==========
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import HuggingFacePipeline

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

**This function initializes and returns an instance of the Language Learning Model (LLM) using the Vicuna-7B model.
The LLM is a Hugging Face pipeline for text generation, specifically designed for language learning tasks.**


In [7]:
def getLLM_model(model_name):
    # Initialize tokenizer for Vicuna-7B model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Initialize model for Vicuna-7B model with custom configurations
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True)
    
    # Set up a text generation pipeline with Vicuna-7B model and tokenizer
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1000, temperature=0, top_p=0.95,)
    
    # Create an instance of the Language Learning Model (LLM) pipeline using the text generation pipeline
    llm = HuggingFacePipeline(pipeline=pipe)
    
    # Return the Language Learning Model (LLM)
    return llm

In [8]:
#Load Documents from Directory using PyPDFLoader.
loader = DirectoryLoader('/kaggle/input/qadocs/', 
                         glob="./*.pdf",
                         loader_cls=PyPDFLoader,
                         show_progress=True,
                         use_multithreading=True)
documents = loader.load()

100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


In [9]:
len(documents)

19

In [10]:
%%time
#Replace Multiple Spaces, Tabs, and Newlines in Document Page Content.
for i in range(len(documents)):
    documents[i].page_content = documents[i].page_content.replace('\t', ' ')\
                                                         .replace('\n', ' ')\
                                                         .replace('       ', ' ')\
                                                         .replace('      ', ' ')\
                                                         .replace('     ', ' ')\
                                                         .replace('    ', ' ')\
                                                         .replace('   ', ' ')\
                                                         .replace('  ', ' ')

CPU times: user 665 µs, sys: 0 ns, total: 665 µs
Wall time: 670 µs


In [11]:
documents[18].page_content


'• Select the language you wish to translate the conversatio n to. • On the Language settings panel, initially two columns are displayed. The first column is of the default language (English) and the second one would of the language that you select for translation. • After you click the eye symbol, the list of elements and attributes is displayed along with the columns of the default language and the translation language. For example, the translations would appear in a three -column format. The first column indicates the elements of the conversation being translated (e.g ., title, description, messages). The second column shows the source language (e.g., English) and the final column displays the translations. • As the author, you have the option to edit the translations by clicking on the corresponding cell and making chan ges. Note that "Message Template" cells will open a new modal with a rich text editor to provide additional features such as adding variables, while the other cells

In [12]:
#This function splits a list of documents into smaller chunks of text based on the specified chunk size and overlap.
#It utilizes the RecursiveCharacterTextSplitter to perform the splitting.
def split_docs(documents, chunk_size=1000, chunk_overlap=200):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

48


### Set up Question Answering (QA) Chain.
#### This function sets up a Question Answering (QA) chain by performing the following steps:
    1. Embedding instructor using HuggingFaceInstructEmbeddings.
    2. Creating a Chroma vector database from the documents.
    3. Persisting the vector database to a directory.
    4. Loading the persisted vector database from a directory.
    5. Creating a retriever from the loaded vector database.
    6. Setting up a RetrievalQA instance with a local language model, retriever, and source document retrieval.

    Parameters:
        documents (list): A list of Document objects.
        instructor_model_name (str): The model name for the instructor embedding, e.g., "hkunlp/instructor-xl".
        persist_directory (str): The directory path to persist the vector database.
        load_directory (str): The directory path to load the persisted vector database.

    Returns:
        qa_chain (RetrievalQA): The configured Question Answering (QA) chain.



In [13]:
def setup_qa_chain(documents, instructor_model_name, persist_directory, load_directory, llm):
    
    # Embed instructor using HuggingFaceInstructEmbeddings
    instructor_embeddings = HuggingFaceInstructEmbeddings(model_name=instructor_model_name, model_kwargs={"device": "cuda"})

    # Create a Chroma vector database from the documents and persist it to a directory
    vectordb = Chroma.from_documents(documents=documents, embedding=instructor_embeddings, persist_directory=persist_directory)
    vectordb.persist()

    # Load the persisted vector database from a directory
    vectordb = Chroma(persist_directory=load_directory, embedding_function=instructor_embeddings)

    # Create a retriever from the loaded vector database
    retriever = vectordb.as_retriever(search_kwargs={"k": 4})

    # Set up a RetrievalQA instance with a local language model, retriever, and source document retrieval
    qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

    # Return the configured Question Answering (QA) chain
    return qa_chain


In [14]:
qa_chain = setup_qa_chain(documents = docs,
               instructor_model_name = "hkunlp/instructor-xl",
               persist_directory = 'db', 
               load_directory = '/kaggle/working/db', 
               llm = getLLM_model(model_name = "eachadea/vicuna-7b-1.1"))


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device": "cuda"})

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
# persist_directory = 'db'

In [17]:
%%time
# ### this takes ~35 min to run
# ### create embeddings and DB
# vectordb = Chroma.from_documents(documents=docs,
#                                  embedding=instructor_embeddings,
#                                  persist_directory=persist_directory)


### persist Chroma database
# vectordb.persist()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.39 µs


In [18]:
# persiste the db to disk
# load_directory = '/kaggle/working/db'
# Now we can load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory=load_directory,
#                   embedding_function=instructor_embeddings)

In [19]:
# retriever = vectordb.as_retriever(search_kwargs={"k": 4})

# qa_chain = RetrievalQA.from_chain_type(llm=local_llm, 
#                                        chain_type="stuff", 
#                                        retriever=retriever, 
#                                        return_source_documents=True)

In [20]:
qa_chain

RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x79e61df49270>, model_id='gpt2', model_kwargs=None, pipeline_kwar

In [21]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [22]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [23]:
%%time
query = "What options can be set up to handle responses in the 'Ask a Person' step?"
llm_ans(query)

 There are several options that can be set up to handle responses in the 'Ask a Person' step in the Krista
Studio. These include:

* Displaying the response as submitted on the right-hand side UI panel for the user who submitted the response
* Sending an audit message to all people involved in the Ask a Person step about a response received
* Disabling the Ask a Person step if the required number of responses has been received
* Displaying the response at the bottom left-hand side corner on the Web Client side
* Setting up options on how to


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 27.9 s, sys: 98 ms, total: 28 s
Wall time: 28.6 s


In [24]:
%%time
query = "What is Krista?"
llm_ans(query)

 Krista is an Intelligent Automation solution developed by Kris ta Software. Our mission is to make technology
that understands people. We are the future of process automation, a conversational app platform with a belief
that conversation can bridge the gap between humans and technology. One of the first questions many people ask
is: What is Intelligent Automation? Krista is technology that understands people. Kris ta is an Intelligent


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 20.3 s, sys: 274 ms, total: 20.5 s
Wall time: 20.5 s


In [25]:
%%time
query =  "What is the mission of Krista Software and their Intelligent Automation solution?"
llm_ans(query)

 Krista Software's mission is to make technology that understands people. They are an Intelligent Automation
solution provider that aims to bridge the gap between humans and technology through their conversational app
platform.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 10.4 s, sys: 14.1 ms, total: 10.5 s
Wall time: 10.5 s


In [26]:
%%time
query =  "What is Intelligent Automation?"
llm_ans(query)

 Intelligent Automation is technology that understands people. It is an Intelligent Automation platform that
makes conventional automation obsolete. With Krista, you can build, deploy, and manage automation to bring
people and systems together.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 11.6 s, sys: 9.7 ms, total: 11.6 s
Wall time: 11.6 s


In [27]:
%%time
query = "In what areas or processes can Krista's Intelligent Automation be applied? Can you provide specific examples?"
llm_ans(query)

 Krista's Intelligent Automation can be applied in various areas or processes such as customer service, sales,
marketing, and HR. For example, in customer service, Krista can be used to automate repetitive tasks such as
answering frequently asked questions, handling complaints, and providing product information. In sales, Krista
can be used to automate tasks such as lead generation,


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 18.2 s, sys: 15.4 ms, total: 18.2 s
Wall time: 18.2 s


In [28]:
%%time
query = "What are the capabilities of Krista's AI and ML engine? "
llm_ans(query)

 Krista's AI and ML engine has the following capabilities: • Developing a classifier model and a regressor
model that is based on random forest algorithms. • Pre-processing of the data includes the following steps for
training vectors: • Accepting different datasets with numerical and categorical properties. • Developing
intelligence. • Accurately predicting outcomes and results. • Ex


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 18.1 s, sys: 18.4 ms, total: 18.1 s
Wall time: 18.1 s


In [29]:
%%time
query = "How does Krista's AI and ML module handle pre-processing of data for training vectors?"
llm_ans(query)

 Krista's AI and ML module pre-processes the data for training vectors by:

* Attributing irrelevant columns that are obvious
* Handling categor


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 8.01 s, sys: 7.71 ms, total: 8.02 s
Wall time: 8.02 s


In [30]:
%%time
query = "What are the key components of the metadata stored by Krista's AI and ML module?"
llm_ans(query)

 The key components of the metadata stored by Krista's AI and ML module are:

* Decision type (Classification / Regression / Anomaly detection)
* Algorithm type (Random Fores t / KNN / SVN / ANN etc.)
*


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 13.8 s, sys: 10.1 ms, total: 13.8 s
Wall time: 13.9 s


In [31]:
%%time
query = "What is the purpose of the Ask Krista AI step in Krista Studio?"
llm_ans(query)

 The purpose of the Ask Krista AI step in Krista Studio is to create a sequential step that uses the Ask
Krista AI feature to make predictions based on the input data and the training model that has been uploaded.
The user sets up the input parameters and the output, and then uploads the training set in the form of a CSV
file. The user can then repeat the


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 18.1 s, sys: 14.4 ms, total: 18.1 s
Wall time: 18.1 s


In [32]:
%%time
query = "How can I set up the input parameters and target attribute for training a model in Krista Studio?"
llm_ans(query)

 To set up the input parameters and target attribute for training a model in Krista Studio, you need to follow
these steps:

1. Go to the Ask Krista AI step in the conversation.
2. Select the Pick One option under the Model panel.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 12.6 s, sys: 23.5 ms, total: 12.6 s
Wall time: 12.6 s


In [33]:
%%time
query = "How does the Confidence threshold in Ask Krista AI affect the efficiency and accuracy of the training model?"
llm_ans(query)

 The Confidence threshold in Ask Krista AI affects the efficiency and accuracy of the training model by
indicating the efficacy and accuracy of how the training model is set up. The Confidence threshold is a
numerical score that ranges from 0 to 100% and


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 12.7 s, sys: 19.9 ms, total: 12.7 s
Wall time: 12.7 s


In [34]:
%%time
query = "Give one example of use of ask krista ai step in studio?"
llm_ans(query)

 The use of the Ask Krista AI step in Krista Studio involves uploading a training dataset in the form of a CSV
file, executing the model, and arriving at a confidence level for the training model and predictions. One
example of the use of this step could be in the healthcare industry, where a dataset of patient health
parameters is uploaded to the Ask Krista AI step, and


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 18 s, sys: 17.1 ms, total: 18 s
Wall time: 18.1 s


In [35]:
%%time
query = "How does the Verify tab in Krista Studio allow for assessing the accuracy and performance of the trained model?"
llm_ans(query)

 The Verify tab in Krista Studio allows for assessing the accuracy and performance of the trained model by
providing various statistical output parameters such as accuracy level, F1 score, and performance report.
These parameters help in evaluating the performance of the trained model and determining its effectiveness in
making predictions.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 14.4 s, sys: 16 ms, total: 14.4 s
Wall time: 14.4 s


In [36]:
%%time
query = "how to use krista ai?"
llm_ans(query)

 To use Krista AI, you need to follow these steps:

1. Go to the Krista Studio and select the conversation in which you want to use the Krista AI step.
2. Click on the "Ask Krista AI" button in the toolbar.
3. In


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 14 s, sys: 23 ms, total: 14 s
Wall time: 14 s


In [37]:
%%time
query = " What is Krista Studio and what is its main function?"
llm_ans(query)

 Krista Studio is an intelligent automation solution developed by Kris ta Software. It is a 'No code' platform
for authoring automation leveraging conversations as natural language. It supports enterprise level security
controls and allows users to perform system-specific actions depending on the extension request made. Krista
Studio's main function is to make technology that understands people and bridge the gap between humans and
technology. It automates processes from collaboration


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 20.2 s, sys: 22 ms, total: 20.2 s
Wall time: 20.2 s


In [38]:
%%time
query = "How does authentication work in the 'Ask a system' step of Krista Studio?"
llm_ans(query)

 In the 'Ask a system' step of Krista Studio, authentication works by either self-authentication or using a
service account. If the 'Have people authenticate with their account' checkbox is not selected, the Web Client
will not authenticate the user and the 'Ask a system' step will not be available. Additionally, when setting
up a specific extension, the authentication method can be set up depending on the third-party system that the
extension


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 20.1 s, sys: 15.8 ms, total: 20.1 s
Wall time: 20.1 s


In [39]:
%%time
query = "What are the limitations of using Krista Studio?"
llm_ans(query)

 Krista Studio has the following limitations:

* It is a 'No code' platform, which means that it does not require any coding skills to use.
* It is a true 'No code' platform, which means that it does not require any coding skills to use.
* It supports enterprise level security controls.
* It has incorporated ways to get to the request.
* It offers various extensions connecting to many third-party systems.
* It has a Global Catalog that has all the ecos


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 23.8 s, sys: 22.4 ms, total: 23.8 s
Wall time: 23.8 s


In [40]:
%%time
query = "What is the purpose of the 'Navigate' tab in Krista Studio and how does it work?"
llm_ans(query)

 The 'Navigate' tab in Krista Studio is used to navigate through the conversation flow and set up the
conversation steps. It allows you to create and configure the conversation steps, such as the 'Ask a person'
step, 'Ask a system' step, and other steps. The 'Navigate' tab is used to set up the conversation flow and
configure the conversation steps, including the fields, variables, and actions that are used in the
conversation. It also


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 21.8 s, sys: 22.9 ms, total: 21.8 s
Wall time: 21.8 s


In [41]:
%%time
query = "What is the Entity tab used for in the 'Ask a system' step of Krista Studio?"
llm_ans(query)

 The Entity tab in the 'Ask a system' step of Krista Studio is used for searching for the step using an
extension. You can select an extension from the drop-down list and then select an area, such as Change
Management or Ticket Management. From there, you can either search for a request or create a new one. You can
also fetch more than 100 records and allow multiple search results.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 20.3 s, sys: 22.4 ms, total: 20.3 s
Wall time: 20.4 s


In [42]:
%%time
query = "How can a new request be added in Krista Studio?"
llm_ans(query)

 To add a new request in Krista Studio, you can follow these steps:

1. Navigate to the Requests tab in the catalog.
2. Click on the "Add Request" button.
3. Fill in the required fields for the new request, such as the name, description, and any other relevant
information.
4. Assign the appropriate ecosystem and extension that the


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 18.6 s, sys: 24.5 ms, total: 18.6 s
Wall time: 18.6 s


In [43]:
%%time
query = "What are some examples of field types that can be added to a request in Krista Studio?"
llm_ans(query)

 Text, Numbers, Location/Date, Composites, and Others.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 4.32 s, sys: 6.87 ms, total: 4.32 s
Wall time: 4.32 s


In [44]:
%%time
query = "What permissions does the Appliance Manager have in Krista Studio by default?"
llm_ans(query)

 The Appliance Manager does not have permissions for KSDK by default.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 4.06 s, sys: 1.06 ms, total: 4.06 s
Wall time: 4.07 s


In [45]:
%%time
query = "What is the purpose of the 'Ask a Person' step in the Krista?"
llm_ans(query)

 The 'Ask a Person' step in Krista is used to initiate a conversation with a person, where the conversation
author can ask a question or make a request to the person. The person can then respond to the conversation
author's request or question. This step is useful for initiating a conversation with a person for the purpose
of gathering information or feedback


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 16.8 s, sys: 16.3 ms, total: 16.8 s
Wall time: 16.8 s


In [46]:
%%time
query = " What options can be set up to handle responses in the 'Ask a Person' step?"
llm_ans(query)

 You can set up options on how to proceed when a person replies to a specific query. You can select one of the
following options: • Take a single response • Wait for all responses • Take the first three responses. You can
also set up an expression from the Expression drop-down list.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 14 s, sys: 14 ms, total: 14 s
Wall time: 14.2 s


In [47]:
%%time
query = "How is the execution flow managed in the 'Ask a Person' step when someone submits a response?"
llm_ans(query)

 The execution flow of the 'Ask a Person' step is managed by the system. When a response is submitted, the
system sends an audit message to all people involved in the step, indicating that a response has been
received. The response is displayed on the right-hand side UI panel for the person who submitted the response.
The system also keeps track of all responses and sends audit messages to all users who have replied. The
system also allows for the setting of rules for how to proceed, such as taking a single response, waiting for
all responses, or taking


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 25.9 s, sys: 25.4 ms, total: 25.9 s
Wall time: 25.9 s


In [48]:
%%time
query = "How can I provide fixed values or scripts in the 'Inform a Person' step?"
llm_ans(query)

 You can provide fixed values or scripts in the 'Inform a Person' step by using the Manage Information step.
In the Manage Information step, you can add a field and configure it to display a fixed value or a script. You
can also add a field to display a specific value based on a previous step. For example, you can add a field to
display the date of birth converted to age.


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 19 s, sys: 13.6 ms, total: 19 s
Wall time: 19 s


In [49]:
%%time
query = "How do I configure the 'Inform a Person' step and provide additional information?"
llm_ans(query)

 To configure the 'Inform a Person' step in Krista Studio, follow these steps:

1. Determine a step that you want to configure in a conversation after the very first step, Person makes a
request or System trigger.
2. Click the Info icon.
3. The configuration options would be displayed.
4. Click Create.
5. On the Add a field panel, diverse options are displayed. You can either search or


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 20.8 s, sys: 22.3 ms, total: 20.8 s
Wall time: 20.9 s


In [50]:
%%time
query = "Can you provide an example of using the 'Inform a Person' step to inform a person's age to their reporting manager?"
llm_ans(query)

 Yes, you can use the 'Inform a Person' step to inform a person's age to their reporting manager. To do this,
you would follow these steps:

1. Set up the Inform a person step.
2. Click the Info icon on the top right-hand corner to prepare and manage information.
3. From the available fields, select Number and rename the number as 'age.'
4. From the date of birth to calculate the age, you would need to build and set up a formula. From the Specify
value drop-down list, select Build a formula.
5. The following UI panel is displayed. You can specify an argument here for the formula.
6. From the available functions to build a formula, search and select the formula


Sources:
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
/kaggle/input/qadocs/Krista QA documents.pdf
CPU times: user 36.1 s, sys: 41 ms, total: 36.1 s
Wall time: 36.2 s
